In [1]:
#@title ##Import libraries!

!pip -q install pyngrok
!pip -q install streamlit

from pyngrok import ngrok

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn import model_selection
from sklearn.model_selection import train_test_split

import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Embedding, Dense, SimpleRNN, LSTM
from keras.wrappers.scikit_learn import KerasClassifier
import keras.optimizers as optimizers
from keras.callbacks import ModelCheckpoint
monitor = ModelCheckpoint('./model.hdf5', 
                          monitor='val_accuracy', 
                          verbose=0, 
                          save_best_only=True, 
                          save_weights_only=False, 
                          mode='auto', 
                          save_freq='epoch')

import gdown

## Set random seed for reproducible results
RAND_SEED = 12
np.random.seed(RAND_SEED)
tf.random.set_seed(RAND_SEED)

## Utils function to combine 23 chunks from the same patient into one big chunk
def prepare_data(eeg_df):
  file_names = eeg_df['Unnamed: 0'].tolist()

  subject_ids = []
  chunk_ids = []
  for fn in file_names:
    subject_ids.append(fn.split('.')[-1])
    chunk_ids.append(fn.split('.')[0])
  subject_ids = list(set(subject_ids))
  assert len(subject_ids) == 500

  sub2ind = {}
  for ind, sub in enumerate(subject_ids):
    sub2ind[sub] = ind

  eeg_combined = np.zeros((500, int(178*23)))
  labels_combined = np.zeros(500)
  labels_chunks = np.zeros((500, 23))
  labels_dict = {}
  for i in range(len(eeg_df)):
    fn = eeg_df.iloc[i]['Unnamed: 0']
    subject_id = fn.split('.')[-1]
    subject_ind = sub2ind[subject_id]

    chunk_id = int(fn.split('.')[0].split('X')[-1])
    start_idx = (chunk_id - 1) * 178
    end_idx = start_idx + 178
    eeg_combined[subject_ind, start_idx:end_idx] = eeg_df.iloc[i].values[1:-1]

    if subject_id not in labels_dict:
      labels_dict[subject_id] = []
    labels_dict[subject_id].append(eeg_df.iloc[i].values[-1])

  for sub_id, labels in labels_dict.items():
    sub_ind = sub2ind[sub_id]
    is_seizure = int(np.any(np.array(labels) == 1))
    labels_combined[sub_ind] = is_seizure
    labels = np.array(labels)
    labels = np.where(labels>1, 0, labels)
    labels_chunks[sub_ind,:] = labels

  return eeg_combined, labels_combined, labels_chunks

     |████████████████████████████████| 747kB 8.2MB/s 
     |████████████████████████████████| 7.0MB 7.1MB/s 
     |████████████████████████████████| 112kB 52.3MB/s 
     |████████████████████████████████| 4.6MB 47.3MB/s 
     |████████████████████████████████| 163kB 51.1MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 122kB 56.5MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.0 which is incompatible.


In [2]:
#@title ## Download our data set!

data_path = 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/Deep%20Dives/AI%20%2B%20Healthcare/Projects%20(Session%206%2B)/Seizure%20Prediction%20/data.csv'
uci_epilepsy = './uci_epilepsy'
gdown.download(data_path, uci_epilepsy, False)

Downloading...
From: https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/Deep%20Dives/AI%20%2B%20Healthcare/Projects%20(Session%206%2B)/Seizure%20Prediction%20/data.csv
To: /content/uci_epilepsy
100%|██████████| 7.64M/7.64M [00:00<00:00, 79.8MB/s]


'./uci_epilepsy'

In [4]:
#@title ## Prepare our data set!

eeg = pd.read_csv(uci_epilepsy)
x, y, y_time_steps = prepare_data(eeg)

# reshape x into (number_of_samples, number_of_time_steps, feature dimension)
x = x.reshape(-1, 23, 178).astype(np.float32) 
np.save('sample', x[30])

# reshape y into (num_of_samples, 1)
y = y.astype(int).reshape(-1,1) 

print('Input x shape: ', x.shape)
print('Label y shape: ', y.shape)

Input x shape:  (500, 23, 178)
Label y shape:  (500, 1)


In [5]:
#@title ## Train our model!

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2020)

# Build a LSTM
model = Sequential()
model.add(LSTM(units=32, return_sequences=True))
model.add(LSTM(units=32, return_sequences=True))
model.add(LSTM(units=32, return_sequences=True))
model.add(LSTM(units=32, return_sequences=True))
model.add(LSTM(units=32, return_sequences=True))
model.add(Dense(1, activation='sigmoid'))

# Compile the LSTM
model.compile(loss='binary_crossentropy',
              optimizer = 'adam', 
              metrics = ['accuracy'])

# Train the LSTM
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, callbacks=[monitor])

# Predict on test data
predictions = model.predict(x_test)
predictions = predictions > 0.5

### END CODE
print('Test accuracy: ', accuracy_score(y_test, predictions[:, -1, :]))
model.save('model.h5')

Epoch 1/20
13/13 [==============================] - 12s 215ms/step - loss: 0.6686 - accuracy: 0.6848 - val_loss: 0.5677 - val_accuracy: 0.7700
Epoch 2/20
13/13 [==============================] - 1s 56ms/step - loss: 0.5089 - accuracy: 0.8141 - val_loss: 0.5491 - val_accuracy: 0.7700
Epoch 3/20
13/13 [==============================] - 1s 56ms/step - loss: 0.4274 - accuracy: 0.8201 - val_loss: 0.4742 - val_accuracy: 0.7948
Epoch 4/20
13/13 [==============================] - 1s 57ms/step - loss: 0.3456 - accuracy: 0.8879 - val_loss: 0.4853 - val_accuracy: 0.8087
Epoch 5/20
13/13 [==============================] - 1s 56ms/step - loss: 0.2560 - accuracy: 0.9342 - val_loss: 0.4808 - val_accuracy: 0.8222
Epoch 6/20
13/13 [==============================] - 1s 57ms/step - loss: 0.2123 - accuracy: 0.9443 - val_loss: 0.4718 - val_accuracy: 0.8243
Epoch 7/20
13/13 [==============================] - 1s 60ms/step - loss: 0.1852 - accuracy: 0.9543 - val_loss: 0.4840 - val_accuracy: 0.8257
Epoch 8/20


In [6]:
#@title ## Build our web application!

%%writefile app.py
import streamlit as st
from joblib import dump, load

import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

model = keras.models.load_model('model.h5')

st.title('Epilepsy Detector') 
uploaded_file = st.file_uploader('Upload Data')
if uploaded_file is not None:
  data = np.load(uploaded_file).reshape(1, 23, 178)
  pred = model.predict(data)
  confidence = pred[:, -1, :][0][0]
  final_pred = pred > 0.5
  final_pred = final_pred[:, -1, :][0][0]
  if final_pred == 0:
    st.write(f'The model is {round(100 - confidence, 2)}% confident that this patient DOES NOT have epilepsy.') 
  else:
    st.write(f'The model is {round(confidence * 100, 2)}% confident that this patient DOES HAVE have epilepsy.') 
  st.line_chart(pd.DataFrame(pred.reshape(-1, 1)))


Writing app.py


In [9]:
#@title ## Run our web application!

public_url = ngrok.connect(port='80')
print(public_url)
!streamlit run app.py >/dev/null

NgrokTunnel: "http://9a6fdeecf080.ngrok.io" -> "http://localhost:80"
2021-03-30 01:24:49.450670: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-30 01:24:50.970018: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-30 01:24:50.970961: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-30 01:24:50.981820: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-03-30 01:24:50.981881: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (1a3e0a987a4d): /proc/driver/nvidia/version does not exist
2021-03-30 01:24:50.982545: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


In [8]:
#@title If you get an error above, run this to reinitialize Streamlit, then try again!
%%writefile ~/.streamlit/config.toml
[server]
port = 80

Writing /root/.streamlit/config.toml
